In [1]:
import warnings
warnings.filterwarnings("ignore")
import json
from lib import config
import numpy as np
import pandas as pd
from lib.country_exposure_summary import Portfolios_Country, Comparison

In [2]:

date_this_qtr = '2023-04-30'
date_last_qtr = '2023-03-31'

port_data = json.load(open(config.INTERNAL_JSON_DIR, 'r'))

def load_port_country(date: str) -> Portfolios_Country:
    port_country = Portfolios_Country(
        date, port_data, config.INTERNAL_DIR.source_dir)
    port_country.load()
    port_country.process()
    port_country.redirect()
    port_country.append()
    return port_country


port_this_qtr = load_port_country(date_this_qtr)
port_last_qtr = load_port_country(date_last_qtr)

comparison = Comparison(
    port_this_qtr, port_last_qtr, config.INTERNAL_DIR.save_dir)
comparison.calculate()

finish loading country exposure for Global Thematic
finish loading country exposure for Global Quantitative
finish loading country exposure for Global Quantitative ESG
finish loading country exposure for Global Fundamental
finish loading country exposure for Global Emerging Markets
finish loading country exposure for Total MSCI World ex Canada
finish calculating Country Active Weight of Global Thematic
finish calculating Country Active Weight of Global Quantitative
finish calculating Country Active Weight of Global Quantitative ESG
finish calculating Country Active Weight of Global Fundamental
finish calculating Country Active Weight of Global Emerging Markets
finish calculating Country Active Weight of Total MSCI World ex Canada
finish compiling benchmark Total MSCI World ex Canada
finish compiling benchmark Global Emerging Markets
finish loading country exposure for Global Thematic
finish loading country exposure for Global Quantitative
finish loading country exposure for Global Quan

In [4]:
comparison.dict['MSCI EM']

,Region,Country,Global Emerging Markets,Global Emerging Markets Change
0,United States of America,United States of America,0.0168,0.0011
1,United States of America,United States of America,0.0168,0.0011
2,China,China,0.0155,0.0081
3,China,China,0.0155,0.0081
4,Developed Asia,Developed Asia,0.0103,-0.0072
5,Developed Asia,Hong Kong,0.0103,-0.0072
6,Emerging Markets,Emerging Markets,-0.0319,0.0008
7,Emerging Markets,Brazil,0.0162,0.0055
8,Emerging Markets,Chile,-0.0055,0.0001
9,Emerging Markets,Colombia,-0.0010,-0.0001


In [3]:
comparison.dict.keys()

dict_keys(['MSCI World ex Canada', 'MSCI EM'])

In [51]:
df = comparison.dict['MSCI World ex Canada']
totals = df.groupby(level=0).sum()
totals.index = pd.MultiIndex.from_product([totals.index, ['Total']], names=['Region', 'Country'])
result = (
    pd.concat([df, totals])
    .reindex(config.COUNTRY_EXPOSURE_ORDER, axis=0, level=0)
    .sort_index(key = lambda x: x.map({'Total': 0}).fillna(1), level=1)
    .sort_index(level=0, sort_remaining=False)
    .reset_index()
    .assign(Country = lambda x: np.where(x['Country'] == 'Total', x['Region'], x['Country']))
    )
result.to_csv('test.csv', index=False)


In [ ]:
# add a total row before the first row of each region
df.groupby('Region').apply(lambda x: x.append(